## 10. Modules and Packages

我需要理解一些 namespace 的作用

### 10.1 理解 Packages of Modules的层级结构

`import`

参考 

- <https://www.learnpython.org/en/Modules_and_Packages>
- <https://stackoverflow.com/questions/7948494/whats-the-difference-between-a-python-module-and-a-python-package>

感觉都没说清楚。简单说来：

1. 每个py文件是一个 module
2. 一个目录有了`__init__.py` 文件，就变成python的一个 package


### 10.2  `from module import *` 控制一下

定义 `__all__` ，选择显示哪些

### 10.3 `import` 相对路径


在这里一定要注意 top路径是什么。 python在执行的时候，文件（或者当前目录）的执行路径，也是 python环境变量的一个。

如果不注意，就很容易产生引用错误。

**一般不要用相对路径。**

### 10.4  Splitting  a Module into Multiple Files

这个不太懂，似乎用处不大


### 10.5 这个问题其实是： 如果一个目录没有 `__init__.py` ，怎么引用目录下的模块？

答案是用 `sys.path.append` ，把这个目录路径给加进来。。


### 10.6 Reloading Modules


In [1]:
import math
import imp
imp.reload(math)

<module 'math' from '/Users/yangdong/.pyenv/versions/anaconda3-5.1.0/envs/DL3/lib/python3.6/lib-dynload/math.cpython-36m-darwin.so'>

### 10.7 运行一个目录，对，就是运行一个目录

在目录下面加上 `__main__.py` 文件，作为运行的入口文件。

如果目录打包成 `.zip` 文件，可以直接运行 zip文件

### 10.8 读取package里面的 datafiles

以前也遇到这个问题，例如在flask里面定义一些模板文件。 那么怎么使用这些模板文件？


```python
import pkgutil
data = pkgutil.get_data(__package__, 'somedata.dat')
```

不明所以。。

### 10.9 黑魔法之 添加环境变量 sys.path

其实也是为了正常import某个特定目录下的modules。 


### 10.10 这个比较常用： 使用模块名（字符串），导入模块

根据参数，选择性，动态的加载模块的一种方式。

In [3]:

import importlib
math = importlib.import_module('math')

print(math.sin(2))

mod = importlib.import_module('urllib.request')
mod

0.9092974268256817


<module 'urllib.request' from '/Users/yangdong/.pyenv/versions/anaconda3-5.1.0/envs/DL3/lib/python3.6/urllib/request.py'>

In [4]:
print(__package__)

None


### 10.11 黑魔法之 远程加载模块。。。

不想看。

大致猜测一下，看上面， 导入的模块，会说明其来源，那么这个模块也可以是远程机器上的。

想不出这样做的应用场景。

### 10.12 Patching Modules on Import

这个也见过， monkey patching。

这里有一点不一样，下面这个例子，可以指定在加载某个模块的时候，干些什么事情，很有意思

In [13]:
# postimport.py

import importlib
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()

    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None
        self._skip.add(fullname)
        return PostImportLoader(self)

class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder

    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)
        self._finder._skip.remove(fullname)
        return module

def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate

# 注意只运行一次啊
if sys.meta_path[0].__class__.__name__ != 'PostImportFinder':
    sys.meta_path.insert(0, PostImportFinder())


In [14]:
sys.meta_path

 _frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,

In [15]:
@when_imported('threading')
def warn_threads(mod):
    print('Threads? Are you crazy?')

if __name__ == '__main__':
    import threading


Threads? Are you crazy?
